In [2]:
pip install spotipy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.2/246.2 kB 2.2 MB/s eta 0:00:002.3 MB/s eta 0:00:01
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 2.2 MB/s eta 0:00:001.9 MB/s eta 0:00:01
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 kB 2.0 MB/s eta 0:00:00 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
launchpadlib 1.10.13 requires testresources, which is not installed.
pandas 1.4.3 requires pytz>=2020.1, but you have pytz 2017.3 which is incompatible.
--- Logging error ---
Traceback (most recent call last):
  File "/home/anthony/.local/lib/python3.8/site-packages/pip/_internal/u

In [457]:
pip install billboard

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement billboard (from versions: none)
ERROR: No matching distribution found for billboard
--- Logging error ---
Traceback (most recent call last):
  File "/home/anthony/.local/lib/python3.8/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/home/anthony/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/home/anthony/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/home/anthony/.local/lib/python3.8/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/home/anthony/.local/lib/python3.8/site-packages/pip/_vendor/rich/segment.py", l

In [20]:
#joaquin
# import packages
import billboard
import pandas as pd
import os
import sys
import json
import spotipy
import webbrowser
import spotipy.util as util
from json.decoder import JSONDecodeError

In [21]:
#joaquin
scope = 'user-read-private user-read-playback-state user-modify-playback-state'

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

token = util.prompt_for_user_token(username = 'anthony_345676543',
                           scope=scope,
                           client_id='36dd9c81d0a543059daebc8fd08a29aa',
                           client_secret='b188c9196c2e43208de4e8a9bdb11570',
                           redirect_uri='http://google.com/')

spotifyObject = spotipy.Spotify(auth=token)

In [23]:
#genre charts to get artist names
#greatest-of-all-time-mainstream-rock-artists
#greatest-alternative-artists
#greatest-country-artists
#greatest-of-all-time-latin-artists
#greatest-of-all-time-pop-songs-artists
#greatest-r-b-hip-hop-artists

In [24]:
#joaquin
def billboard_record_list(rec_type,chartname):
    chart = billboard.ChartData(chartname)
    rec_list =[]
    for item in chart:
        if rec_type == 'artist':
            record=item.artist
        elif rec_type == 'title':
            record=item.title
        elif rec_type == 'rank':
            record=item.rank
        rec_list.append(record)
    return rec_list

In [25]:
#joaquin
artists_searched = set()

In [26]:
#joaquin
artist_list=billboard_record_list('artist','greatest-r-b-hip-hop-artists')

In [47]:
#joaquin
test_list= ['Drake','Mariah Carey','2Pac']
print(test_list)

['Drake', 'Mariah Carey', '2Pac']


In [48]:
#Anthony
user = spotifyObject.current_user()
displayName = user['display_name']
def retrive_artist(artist_list):
    lst_of_lst=[]
    for artist in artist_list:
        searchResults = spotifyObject.search(artist,1,0,"artist")
        artist = searchResults['artists']['items'][0]
        artistID = artist['id']
        albumResult = spotifyObject.artist_albums(artistID,album_type='album', limit=20)
        albumResult = albumResult['items']
        lst_of_lst.append(albumResult)
    album_list = [item for sublist in lst_of_lst for item in sublist]
    return album_list
    

In [49]:
##joaquin
def retrive_album_ids(artists_album_result):
    album_id_list =[]
    for album in artists_album_result:
        album_id_list.append(album['id'])
    return album_id_list

In [50]:
#Anthony
def retrive_albums(album_id_list): 
    track_list=[]
    lst_of_lst=[]
    for album_id in album_id_list:
        trackResults = spotifyObject.album_tracks(album_id)
        trackResults = trackResults['items']
        lst_of_lst.append(trackResults)
    track_list=[item for sublist in lst_of_lst for item in sublist]
    return track_list

In [61]:
#Anthony
def artist_data_frame(artist_data):
    df_artists = pd.DataFrame(columns=['artist_id','artist_name','genres'])
    for artist in artist_data:
        searchResults = spotifyObject.search(artist,1,0,"artist")
        artist = searchResults['artists']['items'][0]
        new_artist_row = pd.Series({'artist_id':artist['id'],'artist_name':artist['name'],'genres':artist['genres']})
        new_artist_row=new_artist_row.to_frame().T
        df_artists = pd.concat([df_artists,new_artist_row],ignore_index=True)
    return df_artists

In [52]:
#Anthony
def album_data_frame(album_res):
    df_albums = pd.DataFrame(columns=['artist_name','album_name','artist_id','album_id'])
    for album in album_res:
        albumID = album['id']
        new_album_row = pd.Series({'artist_name':album['artists'][0]['name'],'album_name':album['name'],'artist_id':album['artists'][0]['id'],'album_id': album['id']})  
        new_album_row=new_album_row.to_frame().T
        df_albums= pd.concat([df_albums,new_album_row], ignore_index=True)
    return (df_albums,album['name'])

In [72]:
#Anthony
def tracks_data_frame(album_info):
    df_tracks = pd.DataFrame(columns=['artist_id','artists_name','song_id','song_name'])
    for track in album_info:
        artists_names =[]
        artists_ids = []
        for artist in track['artists']:
            artists_names.append(artist['name'])
            artists_ids.append(artist['id'])
            new_track_row = pd.Series({'artist_id': artists_ids,'artists_name': artists_names,'song_id': track['id'],'song_name':track['name']})
            new_track_row = new_track_row.to_frame().T
            df_tracks = pd.concat([df_tracks,new_track_row], ignore_index=True)
    return df_tracks
        

In [54]:
#Anthony
artists_info=retrive_artist(test_list)

In [56]:
#Anthony
artists_albums_ids_list=retrive_album_ids(artists_info)

In [58]:
#Anthony
albums_info=retrive_albums(artists_albums_ids_list)

In [62]:
#Anthony
artist_data =artist_data_frame(test_list)
display(artist_data)

,artist_id,artist_name,genres
0,3TVXtAsR1Inumwj472S9r4,Drake,"[canadian hip hop, canadian pop, hip hop, rap,..."
1,4iHNK0tOyZPYnBU7nGAgpQ,Mariah Carey,"[dance pop, pop, r&b, urban contemporary]"
2,1ZwdS5xdxEREPySFridCfh,2Pac,"[g funk, gangster rap, hip hop, rap, west coas..."


In [74]:
#Anthony
artists_album_data=album_data_frame(artists_info)
display(artists_album_data)

,artist_name,album_name,artist_id,album_id
0,Drake,"Honestly, Nevermind",3TVXtAsR1Inumwj472S9r4,3cf4iSSKd8ffTncbtKljXw
1,Drake,"Honestly, Nevermind",3TVXtAsR1Inumwj472S9r4,2hicq6IKmDsPxZV7fJablX
2,Drake,Certified Lover Boy,3TVXtAsR1Inumwj472S9r4,3SpBlxme9WbeQdI9kx7KAV
3,Drake,Certified Lover Boy,3TVXtAsR1Inumwj472S9r4,6sp02aeyiwfX35xRqwNiPv
4,Drake,Dark Lane Demo Tapes,3TVXtAsR1Inumwj472S9r4,6OQ9gBfg5EXeNAEwGSs6jK
5,Drake,Dark Lane Demo Tapes,3TVXtAsR1Inumwj472S9r4,45c1tgTktunRMmfh3WVh8U
6,Drake,Care Package,3TVXtAsR1Inumwj472S9r4,7dqpveMVcWgbzqYrOdkFTD
7,Drake,Care Package,3TVXtAsR1Inumwj472S9r4,6CY70qRxPutN3VKfYhNREa
8,Drake,So Far Gone,3TVXtAsR1Inumwj472S9r4,2podUJIFG8hLfFz7Kqe8yJ
9,Drake,Scorpion,3TVXtAsR1Inumwj472S9r4,1ATL5GLyefJaxhQzSPVrLX


In [73]:
#Anthony
track_data =tracks_data_frame(albums_info)
display(track_data)

,artist_id,artists_name,song_id,song_name
0,[3TVXtAsR1Inumwj472S9r4],[Drake],1kNdtonJXHorsVpWerK8C2,Intro
1,[3TVXtAsR1Inumwj472S9r4],[Drake],1vbn9fEyw1IYhqgZJdu9ZB,Falling Back
2,[3TVXtAsR1Inumwj472S9r4],[Drake],4FIgHqXd4KkOwxrGeLDYM2,Texts Go Green
3,[3TVXtAsR1Inumwj472S9r4],[Drake],1eCFz60zd7mAXgWLapPd9B,Currents
4,[3TVXtAsR1Inumwj472S9r4],[Drake],0nAZGkBGKQCXyaoSJfRhC1,A Keeper
...,...,...,...,...
1737,[1ZwdS5xdxEREPySFridCfh],[2Pac],0iX6mzHk8O0ydBnQJOX9wB,If My Homie Calls
1738,[1ZwdS5xdxEREPySFridCfh],[2Pac],791YTMSeD7SPGWHLIe61ew,Brenda's Got A Baby
1739,[1ZwdS5xdxEREPySFridCfh],[2Pac],30FkHoHz3MA2oBtOcVEBEm,Tha' Lunatic
1740,[1ZwdS5xdxEREPySFridCfh],[2Pac],56sNhTYp0mGC1cXaagTNv1,Rebel Of The Underground
